In [1]:
import selenium
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import warnings
warnings.filterwarnings("ignore")
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re

In [2]:
def cars24_search(search_type,location):
    """Generate a URL for search term with location"""
    
    LOC = {'Delhi': '110001', 'Noida': '201301', 'Gurgoan': '122001', 'Mumbai' : '400001', 'Pune': '411001', 
           'Banglore': '560001','Hyderabad': '500001', 'Chennai': '600001', 'Kolkata': '700001', 'Ahmedabad': '380001' }
    
    search_type = search_type.replace(' ','%20')
    link = f'https://www.cars24.com/buy-used-car?sort=P&search={search_type}&storeCityId=2378&pinId={LOC[location]}'
    
    return link

In [5]:
def cars24_scrapped_data():
    """ 
    Function to scrap datas from cars 24.
    
    """
      
    print("SCRAPPING THE REQUIRED DETAILS")
    driver = webdriver.Chrome(r"/users/pratiksingh/Downloads/chromedriver")    
    driver.maximize_window()
    
    LOC = {'Delhi': '110001', 'Noida': '201301', 'Gurgoan': '122001', 'Mumbai' : '400001', 'Pune': '411001', 
           'Banglore': '560001','Hyderabad': '500001', 'Chennai': '600001', 'Kolkata': '700001', 'Ahmedabad': '380001' }
    
    location = [keys for keys in LOC.keys()]
    
    Car_type= ['SUV', 'Sedan', 'Hatchback','Coupe','Minivan' ,'Luxury Sedan', 'Luxury SUV']
    
    URL = [cars24_search(j,i) for i in location for j in Car_type]
    
    LOCATION = []
    MNF_YEAR = []
    BRAND = []
    MODEL = []
    VARIANT = []
    TRANSMISSION=[]
    driven_KM = []
    NO_OF_owners = []
    fuel_type = []
    PRICE = []


    for i in URL:
        driver.get(i)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        time.sleep(10)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div',{'class' : '_1l4fi'})

        brand_model_name = []
        variant_transmission_name = []
        km_owner_fuel= []
        price = []
    
        for i in results:
            brand_model_name.append(i.find('h2', {'class': "_3FpCg"}).text)
            variant_transmission_name.append(i.find('p', {'class': "cvakB"}).text)
            km_owner_fuel.append(i.find('ul', {'class': "bVR0c"},'li').text)
            price.append(i.find('div', {'class': "_7udZZ"}).text)
            LOCATION.append(driver.find_element_by_xpath("//p[@class='_1V4-O']").text)
    
    
        year = [i.split(' ',2)[0] for i in brand_model_name]
        brand = [i.split(' ',2)[1] for i in brand_model_name]
        #D1 = [i.split(' ',2)[-1] for i in brand_model_name]
        name= [i.split(' ',2)[-1] for i in brand_model_name]

        for i in price:
            PRICE.append(i)

        for i in year:
            MNF_YEAR.append(i)
        for i in brand:
            BRAND.append(i)
    
        #D2 = [i.rsplit(' ',1)[0] for i in variant_transmission_name]
        variant = [i.rsplit(' ',1)[0] for i in variant_transmission_name]
        transmission = [i.rsplit(' ',1)[1] for i in variant_transmission_name]
        for i in variant:
            VARIANT.append(i)
    
    
        for i in name:
            MODEL.append(i)
        for i in transmission:
            TRANSMISSION.append(i)
    
        dummy1 = [i.split('km')[-1]for i in km_owner_fuel]
        kms = [i.split('km')[0] for i in km_owner_fuel]
        kms = [i+'km' for i in kms]

        owners = [i.split('Owner')[0] for i in dummy1]
        owners = [i+'Owner' for i in owners]
        fuel = [i.split('Owner')[-1] for i in dummy1]

        for i in kms:
            driven_KM.append(i)

        for i in owners:
            NO_OF_owners.append(i)

        for i in fuel:
            fuel_type.append(i)

    driver.close()
        
    used_cars_data = pd.DataFrame({"LOCATION" : LOCATION,
                              "MNF_YEAR" : MNF_YEAR,
                              "BRAND" : BRAND,
                              "MODEL" : MODEL,
                              "VARIANT" : VARIANT,
                             "TRANSMISSION": TRANSMISSION,
                              "DRIVEN_KM" : driven_KM,
                              "FUELTYPE" : fuel_type,
                              "NO_OF_OWNERS" : NO_OF_owners,
                              "PRICE" : PRICE
                             })
    
    used_cars_data.to_csv("used_cars-data.csv",index=False) #Creating CSV
    
    return used_cars_data

car_scrap_data = cars24_scrapped_data()
car_scrap_data

SCRAPPING THE REQUIRED DETAILS


,LOCATION,MNF_YEAR,BRAND,MODEL,VARIANT,TRANSMISSION,DRIVEN_KM,FUELTYPE,NO_OF_OWNERS,PRICE
0,New Delhi,2018,Ford,Ecosport,1.5 TDCI TITANIUM PLUS,Manual,"55,399 km",Diesel,1st Owner,"₹8,27,999"
1,New Delhi,2016,Maruti,Vitara Brezza,VDI OPT,Manual,"65,904 km",Diesel,1st Owner,"₹6,55,999"
2,New Delhi,2016,Ford,Ecosport,1.5 TITANIUM TI VCT,Manual,"27,968 km",Petrol,1st Owner,"₹6,81,699"
3,New Delhi,2019,Mahindra,XUV500,W11 (O),Manual,"21,014 km",Diesel,2nd Owner,"₹15,31,399"
4,New Delhi,2013,Mahindra,XUV500,W8 FWD,Manual,"71,669 km",Diesel,1st Owner,"₹5,78,099"
...,...,...,...,...,...,...,...,...,...,...
7342,Ahmedabad,2018,Mahindra,KUV 100 NXT,K2 6 STR,Manual,"36,487 km",Petrol,1st Owner,"₹4,20,299"
7343,Ahmedabad,2017,Maruti,Ertiga,VDI SHVS,Manual,"44,087 km",Diesel,1st Owner,"₹7,84,099"
7344,Ahmedabad,2019,Mahindra,XUV500,W7 FWD,Manual,"57,455 km",Diesel,1st Owner,"₹11,57,799"
7345,Ahmedabad,2017,Ford,Ecosport,1.5 TITANIUMTDCI OPT,Manual,"59,877 km",Diesel,2nd Owner,"₹7,56,199"
